In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline


In [6]:
info = pd.read_csv('./wecode.csv', encoding='utf-8', sep=',')

In [7]:
info.head()


,id,assignment_id,problem_id,user_id,is_final,status,pre_score,coefficient,file_name,language_id,created_at,updated_at
0,4,0,3,4,1,SCORE,10000,100,solution-0,2,2020-10-30 10:58:11,2020-10-30 10:58:19
1,5,0,4,4,1,SCORE,10000,100,solution-1,2,2020-10-30 11:07:09,2020-10-30 11:07:17
2,6,0,5,4,0,Compilation Error,0,100,solution-2,2,2020-10-30 11:08:01,2020-10-30 11:23:36
3,7,0,5,4,0,Compilation Error,0,100,solution-3,2,2020-10-30 11:11:41,2020-10-30 11:11:42
4,8,0,5,4,0,Compilation Error,0,100,solution-4,2,2020-10-30 11:15:06,2020-10-30 11:15:07


In [8]:
sorted_info = info.sort_values(by=["user_id","problem_id"], ascending=True)
sorted_info.head()

,id,assignment_id,problem_id,user_id,is_final,status,pre_score,coefficient,file_name,language_id,created_at,updated_at
13940,14008,0,81,1,1,Compilation Error,0,100,solution-1791,2,2020-11-08 16:50:50,2020-11-08 16:50:50
84860,84971,0,370,1,1,SCORE,10000,100,solution-2367,2,2020-12-01 01:59:52,2020-12-01 02:00:05
0,4,0,3,4,1,SCORE,10000,100,solution-0,2,2020-10-30 10:58:11,2020-10-30 10:58:19
1,5,0,4,4,1,SCORE,10000,100,solution-1,2,2020-10-30 11:07:09,2020-10-30 11:07:17
2,6,0,5,4,0,Compilation Error,0,100,solution-2,2,2020-10-30 11:08:01,2020-10-30 11:23:36


In [11]:
info.describe().T

,count,mean,std,min,25%,50%,75%,max
id,264606.0,132535.074908,76605.695631,4.0,66262.25,132413.5,198566.75,266671.0
assignment_id,264606.0,186.732429,109.768713,0.0,90.00,178.0,286.00,384.0
problem_id,264606.0,257.958074,158.359795,1.0,123.00,298.0,348.00,698.0
user_id,264606.0,935.002147,514.246324,1.0,497.00,925.0,1393.00,1868.0
is_final,264606.0,0.387349,0.487146,0.0,0.00,0.0,1.00,1.0
pre_score,264606.0,5596.454559,4393.183730,0.0,0.00,7000.0,10000.00,10000.0
language_id,264606.0,2.000317,0.025195,2.0,2.00,2.0,2.00,4.0


In [13]:
userSubmit = info.groupby(['user_id']).sum()
userSubmit.head(20)

,id,assignment_id,problem_id,is_final,pre_score,language_id
user_id,,,,,,
1,98979,0,451,2,10000,4
4,2025618,0,8518,38,466000,108
5,317619,613,748,5,30000,12
6,11670135,673,34030,88,965637,260
7,792177,0,1700,2,18843,10
11,6097888,7798,17855,12,317268,100
14,921840,1458,3176,7,130428,44
15,6719541,3143,22154,52,629946,202
20,233334,0,1730,12,120000,26


In [12]:
info.describe

<bound method NDFrame.describe of             id  assignment_id  problem_id  user_id  is_final  \
0            4              0           3        4         1   
1            5              0           4        4         1   
2            6              0           5        4         0   
3            7              0           5        4         0   
4            8              0           5        4         0   
...        ...            ...         ...      ...       ...   
264601  266667              0          25     1796         0   
264602  266668              0          25     1796         0   
264603  266669              0          25     1796         1   
264604  266670              0         150      255         1   
264605  266671             21          24     1568         0   

                   status  pre_score coefficient      file_name  language_id  \
0                   SCORE      10000         100     solution-0            2   
1                   SCORE      10000 